In [1]:
import csv
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly
import os
import tqdm
import json
from collections import defaultdict
import json_lines
import datetime
import matplotlib.pyplot as plt
from statistics import median
from ipywidgets import interact

# 파일 불러오기

In [2]:
# 성지 후보군 기사들
with open('../pilgrim data/result/pilgrim_candidates.json') as f:
    cands = json.load(f)

# sid 별 매칭 정보가 담긴 파일
with open('../pilgrim data/sid_to_cat.csv', 'r', encoding = 'utf-8') as g:
    reader = csv.reader(g)
    sid_to_cat = {line[0]: line[1] for line in reader}
    del sid_to_cat['\ufeffsid']

In [42]:
i = 1
for cand in [a for a in cands]:
    cand = cand[1]
    print('{}. {}'.format(i, cand['title']))
    print('URL: {}'.format(cand['url']))
    scores.append(score)
    i += 1

1. 중증장애인 부부의 '아름다운 기부'
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0005964326
2. 부안여고 선플누리단 'YES 2011 사진 전시회' 21일까지 의회 전시
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0004130332
3. '궁금한 이야기Y' 학교 폭력 피해자를 성추행 가해자로 바꾼 쪽지 한 장
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=213&aid=0000523205
4. [단독] 최태원 내연녀 김 씨, 어머니 대신 금감원 출석시켜
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=056&aid=0010276883
5. 安 "청년, 투표 안하니 지원법 저조…어느 당 찍든 투표해야"
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0007125210
6. '데뷔' 방탄소년단 "롤모델은 빅뱅, 살아남겠다"
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=108&aid=0002225969
7. 국산 경유차, 연비 경쟁에 '폐질환 유발' 오염 물질 외면
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=437&aid=0000055288
8. 뇌성마비 부부 애타는 사연 “눈으로 안을 수 밖에 없는 딸…”
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=

# 성지 후보 목록의 기사들을 카테고리 별로 분류

In [3]:
pilg_categories = defaultdict(int)
for cand in cands:
    data = cand[1]
    cats = []
    for sid in data['sid1s']:
        cats.append(sid_to_cat[sid])
    pilg_categories[','.join(cats)] += 1

ordered_cat =  sorted(
    list(pilg_categories.items()), key=lambda x: x[1], reverse=True)
ordered_cat

[('사회', 275),
 ('연예', 200),
 ('정치', 87),
 ('생활/문화', 79),
 ('경제', 30),
 ('IT/과학', 28),
 ('세계', 28),
 ('분류안됨', 15),
 ('사회,생활/문화', 8),
 ('정치,사회', 5),
 ('경제,사회', 5),
 ('오피니언', 5),
 ('정치,경제', 4),
 ('세계,연예', 2),
 ('사회,IT/과학', 2),
 ('생활/문화,세계', 2),
 ('사회,생활/문화,오피니언', 1),
 ('오피니언,연예', 1),
 ('생활/문화,연예', 1),
 ('경제,생활/문화', 1),
 ('생활/문화,IT/과학', 1),
 ('경제,사회,생활/문화,연예', 1),
 ('사회,생활/문화,IT/과학', 1),
 ('생활/문화,오피니언', 1),
 ('경제,IT/과학', 1),
 ('사회,생활/문화,연예', 1),
 ('사회,연예', 1),
 ('경제,연예', 1)]

# Timeline CDF로 각 성지를 표현해보기
## 직접적인 성지 언급이 있는 기사: 174건

### 각 뉴스 별 CDF 확인

In [4]:
pilg_by_cat = defaultdict(list)
for cand in cands:
    data = cand[1]
    cats = []
    for sid in data['sid1s']:
        cats.append(sid_to_cat[sid])
    pilg_by_cat[','.join(cats)].append(data)

        
@interact (idx = (0,786))
def show_cdf (idx = 0):
    fig = go.Figure()
    target = cands[idx][1]
    
    #기사정보 표시하기
    print('Title: {}'.format(target['title']))
    print('Published: {}'.format(target['timestamp']))
    print('URL: {}'.format(target['url']))
    print('직접적인 성지 언급: {} \n'.format(target['is_pilgrim']))
    
    timeline = target['timeline']
#     for k,v in timeline.items():
#         print('{} : {} comments'.format(k,v))
    
    published_date = datetime.datetime.strptime(target['timestamp'][:10], '%Y-%m-%d')
    dates = list(map(lambda x: datetime.datetime.strptime(x, '%Y%m%d'), list(target['timeline'].keys())))
    cnts = np.cumsum(list(target['timeline'].values())) / sum(list(target['timeline'].values()))
    dates.insert(0, published_date)
    cnts = np.insert(cnts, 0, 0)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = dates, y = cnts, mode='markers+lines'))
    fig.update_yaxes(range = [0,1])
    # fig.update_xaxes(range=[published_date,
    #                                datetime.datetime(2019, 12, 31)])
    fig.update_xaxes(range=[published_date,
                                   dates[-1]])
    fig.show()

interactive(children=(IntSlider(value=0, description='idx', max=786), Output()), _dom_classes=('widget-interac…

### 카테고리 별로 CDF 묶어서 확인.

In [7]:
@interact (idx = (0,19))
def show_cdf_by_category (idx = 0):
    fig = go.Figure()
    targets = pilg_by_cat[ordered_cat[idx][0]]
    
    for target in targets:
        timeline = target['timeline']
        published_date = datetime.datetime.strptime(target['timestamp'][:10], '%Y-%m-%d')
        dates = list(map(lambda x: datetime.datetime.strptime(x, '%Y%m%d'), list(target['timeline'].keys())))
        cnts = np.cumsum(list(target['timeline'].values())) / sum(list(target['timeline'].values()))
        dates.insert(0, published_date)
        cnts = np.insert(cnts, 0, 0)
        fig.add_trace(go.Scatter(x = dates, y = cnts, mode='markers+lines', name = target['title']))
    
    fig.update_yaxes(range = [0,1])
    fig.update_xaxes(range=[datetime.datetime(2007,1,1),
                                   datetime.datetime(2019,12,31)])
    fig.update_layout(title = "카테고리: {} / 기사 수: {}개".format(ordered_cat[idx][0], ordered_cat[idx][1]))
    fig.show()

interactive(children=(IntSlider(value=0, description='idx', max=19), Output()), _dom_classes=('widget-interact…

# Pilgrim Score
- 발행 이후 N 일 뒤의 댓글 수가 많을수록
- 댓글이 달린 기간이 길수록
- 최근에 달린 댓글이 많을수록
- 단발성, 집중성 성지가 아닐 수록

    더 좋은 성지에 가깝도록 점수를 구체화.
    
3개의 스코어를 만들어 그것을 합산하여 각 기사 별 점수를 뽑아내도록 함.

- OLD_SCORE
    - 각 댓글이 기사 생성일로부터 오래 될수록 가중치를 부여하여 매기는 점수
    - 각 기사 별로, 1년 이상된 댓글들의 평균 나이를 계산.
    
- RECENT_SCORE
    - 최근에 작성된 댓글로 이루어진 성지일수록 높은 점수를 받도록 구성. 
    - 같은 달에 쓰여진 댓글의 경우 같은 가중치를 갖도록. (12월 30일과 12월 1일은 거시적인 관점에서 큰 차이 없게.)
    
- CONCENTRATED_SCORE
    - 특정 기간에 집중되어 만들어진 성지를 찾고자 함.
    - 이게 너무 높으면 좋은 성지라고 보기는 어렵도록 설정.
    - 이 값이 클 때에만 전체 스코어에 큰 영향을 끼치도록, 작을 수록 별 차이 없도록 함.
    - 이 값이 특정 값 이상일 경우 전부 같은 값을 갖도록 함.
    - 레벨화하여 같은 레벨에서는 소수점 값이 같더라도 같은 영향력을 가지도록 함.

In [6]:
def total_score(info):
    '''
    input: 각 기사에 대한 정보 (timeline, timestamp, etc)
    output: 각 기사 별 pilgrim score

    OLD_SCORE: 각 댓글이 기사 생성일로부터 오래 될수록 가중치를 부여하여 매기는 점수
    RECENT_SCORE: 최근에 작성된 댓글로 이루어진 성지일수록 높은 점수를 받도록 구성.
                  같은 달에 쓰여진 댓글의 경우 같은 가중치를 갖도록. (12월 30일과 12월 1일은 거시적인 관점에서 큰 차이 없게.)
    CONCENTRATED_SCORE: 특정 기간에 집중되어 만들어진 성지를 찾고자 함.
                        이게 너무 높으면 좋은 성지라고 보기는 어렵도록 설정.
                        이 값이 클 때에만 전체 스코어에 큰 영향을 끼치도록, 작을 수록 별 차이 없도록 최종 식에서 설정.
    '''
    N = 365
    published_date = datetime.datetime.strptime(
        info['timestamp'][:10], '%Y-%m-%d')
    timeline = info['timeline']
    dates = list(map(lambda x: (x, datetime.datetime.strptime(
        x, '%Y%m%d')), list(timeline.keys())))
    length = (dates[-1][1] - published_date).days

    old_score = 0
    recent_score = 0
    days = []
    pilg_count = 0

    for string, date in dates:
        diff = (date - published_date).days
        if diff >= N:
            old_score += diff * timeline[string]
            recent_score += 1 / \
                ((datetime.datetime(2019, 12, 31) -
                  date).days // 30) * timeline[string]
            pilg_count += timeline[string]
            for i in range(timeline[string]):
                days.append(diff)

    os = old_score / pilg_count / 365
    rs = recent_score / pilg_count * 100  # 절대적인 크기를 맞추어주기 위해서
    amount = len(days)
    # 집중도를 계산하는 데에 존재하는 아웃라이어들을 제거.
    days = days[5:-
                5] if amount > 50 else days[int(amount * .1): int(amount * .9)]
    cs = 10e-3 if np.std(days) == 0 else np.std(days)
    cs = 5 if 1/cs > 5 else 1/cs
    cs = int(cs+1)

    return os, rs, cs, (os * rs) / cs * pilg_count
#     return os, rs, cs, (os * rs) * cs * pilg_count


def num_pilg_comments(info):
    N = 365
    published_date = datetime.datetime.strptime(
        info['timestamp'][:10], '%Y-%m-%d')
    timeline = info['timeline']
    dates = list(map(lambda x: (x, datetime.datetime.strptime(
        x, '%Y%m%d')), list(timeline.keys())))
    length = (dates[-1][1] - published_date).days
    pilg_count = 0
    for string, date in dates:
        diff = (date - published_date).days
        if diff >= N:
            pilg_count += timeline[string]

    return pilg_count

## Pilgrim Score 순서로 기사 정렬

In [10]:
infos = [cand[1] for cand in cands if cand[1]]
scored_candidates = sorted(list(map(lambda x: (total_score(x), num_pilg_comments(x),
                                               x['title'], x['url'], x['is_pilgrim']), infos)),
                          key = lambda x: x[0][3], reverse = True)

i = 1
for (os, rs, cs, ts), num, title, url, TF in scored_candidates:
    print('{}. {}  {}'.format(i, title, TF))
    print('num: {}  old: {:.3f}yr  recent: {:.3f}  tight_level: {}'.format(num, os, rs, cs))
    print('score: {:.3f}'.format(ts))
    print('URL: {}'.format(url))
    i += 1

1. 중증장애인 부부의 '아름다운 기부'  NO
num: 2747  old: 4.547yr  recent: 3.414  tight_level: 1
score: 42636.433
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0005964326
2. '데뷔' 방탄소년단 "롤모델은 빅뱅, 살아남겠다"  YES
num: 651  old: 5.478yr  recent: 8.712  tight_level: 1
score: 31066.298
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=108&aid=0002225969
3. SM, '춤짱' 포함 男중고생 5인조 신인그룹 공개  YES
num: 331  old: 9.524yr  recent: 4.601  tight_level: 1
score: 14504.160
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=108&aid=0001947188
4. 뇌성마비 부부 애타는 사연 “눈으로 안을 수 밖에 없는 딸…”  NO
num: 583  old: 9.414yr  recent: 2.608  tight_level: 1
score: 14314.321
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=143&aid=0000061404
5. 李측 "朴이 대통령 되면 최태민 일가 국정농단"  YES
num: 515  old: 9.478yr  recent: 2.836  tight_level: 1
score: 13841.745
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0000458405
6. 목욕탕 기부 천사

num: 41  old: 1.266yr  recent: 3.792  tight_level: 1
score: 196.825
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=020&aid=0002988433
404. 국회의원 연금법 통과에 누리꾼 '부글부글'  NO
num: 139  old: 1.017yr  recent: 1.389  tight_level: 1
score: 196.487
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002015062
405. 통역도 깜짝 놀란 朴대통령의 '외국어 실력'  NO
num: 31  old: 4.677yr  recent: 4.033  tight_level: 3
score: 194.920
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=015&aid=0002840600
406. 정말 쓸데없는 물건 “정체가 뭔가요?” 궁금증 폭발  YES
num: 27  old: 2.991yr  recent: 2.412  tight_level: 1
score: 194.824
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=382&aid=0000042628
407. [단독]국회 농수산위, 피감기관 돈으로 무더기 출장  NO
num: 26  old: 4.620yr  recent: 4.835  tight_level: 3
score: 193.605
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0005342197
408. 항문에 샤워호스를 꽂아놓고 도대체 무슨 짓을 하는거야?  NO
num: 22  old: 1.677y

778. ‘딸랑’ 장기자랑인가 했더니만  NO
num: 24  old: 2.092yr  recent: 0.935  tight_level: 3
score: 15.641
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=036&aid=0000019319
779. 공주 욕하는 자, 공포에  떨지니  NO
num: 45  old: 1.384yr  recent: 1.408  tight_level: 6
score: 14.611
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=308&aid=0000008188
780. 인천 어린이집서도 보육교사가 원아 폭행  NO
num: 25  old: 1.700yr  recent: 1.664  tight_level: 5
score: 14.144
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0006238211
781. '오만과 편견' 백진희, 노주현과 정면 승부 '15년전 한별 살해범 체포'  YES
num: 30  old: 1.263yr  recent: 2.222  tight_level: 6
score: 14.033
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=016&aid=0000601621
782. '동물농장' 멸종위기 바다코끼리의 한국 적응기  NO
num: 34  old: 1.797yr  recent: 1.341  tight_level: 6
score: 13.657
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=213&aid=0000170514
783. 정용화 금발 변신, 지드래곤보다 어울려 ‘컴백 기대’  NO

# 번외

## (가장 최근 댓글) - (발행일) 순서대로 정렬

In [107]:
# 최장기간 성지 찾기
def lifetime (info):
    published_date = datetime.datetime.strptime(info['timestamp'][:10], '%Y-%m-%d')
    timeline = info['timeline']
    dates = list(map(lambda x: (x, datetime.datetime.strptime(x, '%Y%m%d')), list(timeline.keys())))
    length = (dates[-1][1] - published_date).days
    
    return length
    
infos = [cand[1] for cand in cands]
scored_candidates = sorted(list(map(lambda x: (lifetime(x), x['title'], x['url'], x['is_pilgrim']), infos)),
                          key = lambda x: x[0], reverse = True)

i = 1
for score, title, url, TF in scored_candidates:
    print('{}. {} / {}'.format(i, title, TF))
    print('기간: {}년 {}일'.format(score//365, score%365))
    print('URL: {}'.format(url))
    i += 1

1. <붕어를찾아>‘배스 때문에…’ 낚이면 ‘월척’ / NO
기간: 12년 78일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=021&aid=0000188855
2. 베네수엘라 국민에게 길을 묻자 / YES
기간: 12년 59일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=036&aid=0000014435
3. 김창렬 "무서워 어머니 간이식 못해 드려" / YES
기간: 12년 49일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=111&aid=0000062455
4. 홍준표 "대북 쌀 지원과 핵문제는 연계하지 말아야" / YES
기간: 12년 45일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=079&aid=0000152175
5. 백주대낮에 돼지 능지처참 / NO
기간: 12년 29일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=022&aid=0000229162
6. <종합>"박근혜는 최태민과 딸의 꼭두각시"...의혹제기 / YES
기간: 12년 23일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0000457092
7. "노무현이 민주화 추락시켰다"<BR>"개발독재자 딸이 지지받는데" / NO
기간: 11년 356일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=047&aid=0000098546
8. 이력서 ‘튀기’ 경쟁 메일 제목부터 시작 / YES
기간: 11년 

기간: 2년 184일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=022&aid=0003031792
567. "여기자만 질문하세요" 오바마 올 마지막 기자회견 '파격' / NO
기간: 2년 183일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=011&aid=0002616421
568. 공무원 양성평등채용, 남성이 여성보다 혜택 더 본다 / NO
기간: 2년 180일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=277&aid=0003806983
569. 문재인 대표, “교과서 집필진, 알 권리가 있다”, 공개 요구 / NO
기간: 2년 179일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=032&aid=0002649153
570. 불길 속 택시기사 살린 고교생에 '용감한 시민상' / NO
기간: 2년 178일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0005869133
571. 중3 프로게이머, 셧다운제 때문에 국제 대회 포기(종합) / NO
기간: 2년 175일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0005873787
572. 문재인 트위터에 음란물 이미지…"해킹" vs "실수" / YES
기간: 2년 172일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=448&aid=0000181482
573. 지코가 폭로한 정준영 '황금폰'의 진실, "같이

## 총 댓글 수 순서대로 정렬

In [117]:
infos = [cand[1] for cand in cands]
scored_candidates = sorted(list(map(lambda x: (sum(list(x['timeline'].values())), x['title'], x['url'], x['is_pilgrim']), infos)),
                          key = lambda x: x[0], reverse = True)

i = 1
for score, title, url, TF in scored_candidates:
    print('{}. {} / {}'.format(i, title, TF))
    print('댓글 수: {}개'.format(score))
    print('URL: {}'.format(url))
    i += 1

1. “친아빠에게 10년 동안 성폭행 당한 두 아들, 아이들을 살려주세요” / YES
댓글 수: 167763개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=005&aid=0000789116
2. "보.이.콧"…제이에스티나, 中에서 송혜교 역풍 / NO
댓글 수: 127531개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=433&aid=0000016711
3. 서러운 떡볶이 아줌마 / YES
댓글 수: 21724개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=003&aid=0001972907
4. [단독] 태연·백현, 오픈카 데이트 포착…SM 사내커플 탄생 (종합) / NO
댓글 수: 17382개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=433&aid=0000004778
5. "순천 변사체, DNA·지문으로 유병언 확인"(종합) / YES
댓글 수: 15125개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0007026400
6. [단독] 최태원 내연녀 김 씨, 어머니 대신 금감원 출석시켜 / YES
댓글 수: 14486개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=056&aid=0010276883
7. 법원, 2살 아기 던져 살해한 발달장애아에 무죄선고 / NO
댓글 수: 13072개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0007602613


댓글 수: 52개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=005&aid=0000801621
535. 윤아, 이번에는 '걸어다니는 인형' '막 찍어도 화보' / NO
댓글 수: 52개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=213&aid=0000165095
536. NS홈쇼핑, '칭찬쿠폰' 제도 눈길…도입 한달만에 택배사원 3222명 칭찬 / NO
댓글 수: 52개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=015&aid=0003016008
537. ‘대한민국 최초의 남성인권 운동가 성재기’ 19일 출간 / NO
댓글 수: 52개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0003244565
538. [T포토] 구구단 미나 '살랑살랑 애교 댄스' / NO
댓글 수: 52개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=213&aid=0000885901
539. [풀영상]홍준표 "안상수 대표, 독선이 도를 넘었다" / YES
댓글 수: 51개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=214&aid=0000149382
540. SM 엑소 10번째 멤버 수호 공개..보컬+연기+비주얼 3박자 '합격' / NO
댓글 수: 51개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=213&aid=0000181277
541. 소녀시대, 통닭 먹고 싶어 어쩌니? / NO
댓글